In [1]:
import numpy as np

def read_mnist_images(filename):
    with open(filename, 'rb') as f:
        f.read(16)  # skip header
        images = np.fromfile(f, dtype=np.uint8).reshape(-1, 28, 28)
    return images

def read_mnist_labels(filename):
    with open(filename, 'rb') as f:
        f.read(8)  # skip header
        labels = np.fromfile(f, dtype=np.uint8)
    return labels

# 讀取數據
train_images = read_mnist_images('train-images.idx3-ubyte_')
train_labels = read_mnist_labels('train-labels.idx1-ubyte_')
test_images = read_mnist_images('t10k-images.idx3-ubyte_')
test_labels = read_mnist_labels('t10k-labels.idx1-ubyte_')


In [5]:
#先驗機率

def prior(data_labels):
    # 確保標籤是單維度的數組
    data_labels = data_labels.flatten()

    # 初始化計數陣列
    counts = np.zeros(10, dtype=int)

    # 手動計算每個數字的出現次數
    for label in data_labels:
        if 0 <= label < 10:
            counts[label] += 1

    # 計算總標籤數
    total_labels = len(data_labels)

    # 計算每個類別的先驗機率
    prior = counts / total_labels
    return prior

prior = prior(train_labels)


In [30]:
def map_to_32_bins(data_images):
    discrete_train_images = (data_images // 8).astype(int)
    return discrete_train_images

def compute_pixel_probabilities(discrete_train_images, train_labels):
    digit_pixel_counts = np.zeros((10, 32, 28, 28))
    class_counts = np.zeros(10)

    for i in range(discrete_train_images.shape[0]):
        label = train_labels[i]
        for x in range(28):
            for y in range(28):
                pixel_value = discrete_train_images[i, x, y]
                digit_pixel_counts[label, pixel_value, x, y] += 1
        class_counts[label] += 1

    epsilon = 1e-10
    pixel_probabilities = digit_pixel_counts / (class_counts[:, None, None, None] + epsilon)
    return pixel_probabilities

def compute_gaussian_pixel_probabilities(discrete_train_images, train_labels):
    num_classes = 10
    image_size = 28  # 假設影像為 28x28
    num_pixel_values = 256  # 假設像素值範圍為 0-255
    epsilon = 1e-6  # 防止標準差為 0 的穩定項

    # 初始化均值和標準差
    mu = np.zeros((num_classes, image_size, image_size))
    sigma = np.zeros((num_classes, image_size, image_size))

    # 計算每個類別的均值和標準差
    for j in range(num_classes):
        class_images = discrete_train_images[train_labels == j]
        mu[j] = np.mean(class_images, axis=0)  # 計算均值
        sigma[j] = np.std(class_images, axis=0) + epsilon  # 計算標準差並加入穩定項

    # 預先展開像素值範圍為 [0, 255]
    pixel_values = np.arange(num_pixel_values).reshape(-1, 1, 1)  # shape: (256, 1, 1)

    # 計算條件概率 (高斯 PDF)
    pixel_probabilities = np.zeros((num_classes, num_pixel_values, image_size, image_size))
    for j in range(num_classes):
        coeff = 1 / (np.sqrt(2 * np.pi) * sigma[j])  # 前項係數
        exponent = -0.5 * ((pixel_values - mu[j]) ** 2) / (sigma[j] ** 2)  # 指數部分
        pixel_probabilities[j] = coeff * np.exp(exponent)

    return pixel_probabilities

def compute_posterior(data_images, prior_probabilities, pixel_probabilities):
    num_images = data_images.shape[0]
    # 儲存每張圖像的後驗概率
    posteriors = np.zeros((num_images, 10))

    for i in range(num_images):
        for j in range(10):
            # 計算條件機率乘積的對數
            log_likelihood = 0.0
            for x in range(28):
                for y in range(28):
                    pixel_value = data_images[i, x, y]
                    log_likelihood += np.log(pixel_probabilities[j, pixel_value, x, y] + 1e-10)  # 加上小常數以避免對數為無限大
            
            # 計算後驗概率的對數
            posteriors[i, j] = np.log(prior_probabilities[j] + 1e-10) + log_likelihood  # 加上小常數以避免對數為無限大

    return posteriors


In [39]:
# 離散模式

discrete_train_images = map_to_32_bins(train_images)
discrete_test_images =  map_to_32_bins(test_images)
pixel_probabilities = compute_pixel_probabilities(discrete_train_images, train_labels)

posteriors = compute_posterior(discrete_test_images, prior, pixel_probabilities)

# 預測類別是後驗概率最大的那一個
predicted_labels = np.argmax(posteriors, axis=1)
# 計算正確預測的數量
correct_predictions = np.sum(predicted_labels == test_labels)
# 計算準確率
accuracy = correct_predictions / len(test_labels)
print("Accuracy:", accuracy)  

#test
for i in range(2):
    print(f"Max index: {predicted_labels[i]}, True label: {test_labels[i]}")
  

Accuracy: 0.8506
Max index: 7, True label: 7
Max index: 2, True label: 2


In [41]:
#連續模式

# 計算高斯像素條件概率、均值和標準差
pixel_probabilities = compute_gaussian_pixel_probabilities(train_images, train_labels)
# 計算後驗概率
posteriors = compute_posterior(test_images, prior, pixel_probabilities)
# 找到最大值的索引
predicted_labels = np.argmax(posteriors, axis=1)

# 計算正確預測的數量
correct_predictions = np.sum(predicted_labels == test_labels)
# 計算準確率
accuracy = correct_predictions / len(test_labels)
print("Accuracy:", accuracy)

#test
for i in range(2):
    # 印出最大值的索引
    print(f"Max index: {predicted_labels[i]}, True label: {test_labels[i]}")


Accuracy: 0.4243
Max index: 9, True label: 7
Max index: 6, True label: 2


Beta-Binomial Conjugate

In [6]:
# 初始化 a 和 b
a, b = 10, 1

# 定義 lines 範例資料
lines = [
    "0101010101001011010101",
    "0110101",
    "010110101101",
    "0101101011101011010",
    "111101100011110",
    "101110111000110",
    "1010010111",
    "11101110110",
    "01000111101",
    "110100111",
    "01101010111"
]

# 計算階乘的函數
def factorial(num):
    result = 1
    for i in range(1, num + 1):
        result *= i
    return result

# 計算 p 的 x 次方
def power(p, x):
    result = 1
    for _ in range(x):
        result *= p
    return result

# 使用 for 迴圈逐行處理
for index, line in enumerate(lines):
    # 先處理當前的 case
    print(f"case {index + 1}:", line.strip())
    print("Beta prior:", "a =", a, "b =", b)

    # 計算 1 的次數 (正面的次數)
    count = 0
    for i in line.strip():
        if i == "1":
            count += 1

    # Beta 後驗參數更新
    x = count
    n = len(line.strip())
    p = x / n

    # 條件機率 p(x|n,p)
    likelihood = (factorial(n) / (factorial(x) * factorial(n - x))) * power(p, x) * power(1 - p, n - x)
    print("Likelihood:", likelihood)

    # 更新 Beta 分佈的後驗參數
    a, b = a + x, b + (n - x)
    print("Beta posterior:", "a =", a, "b =", b)
    print()  # 用來區隔每個 case 的輸出


case 1: 0101010101001011010101
Beta prior: a = 10 b = 1
Likelihood: 0.16818809509277344
Beta posterior: a = 21 b = 12

case 2: 0110101
Beta prior: a = 21 b = 12
Likelihood: 0.29375515303997485
Beta posterior: a = 25 b = 15

case 3: 010110101101
Beta prior: a = 25 b = 15
Likelihood: 0.2286054241794335
Beta posterior: a = 32 b = 20

case 4: 0101101011101011010
Beta prior: a = 32 b = 20
Likelihood: 0.18286870706509095
Beta posterior: a = 43 b = 28

case 5: 111101100011110
Beta prior: a = 43 b = 28
Likelihood: 0.2143070548857833
Beta posterior: a = 53 b = 33

case 6: 101110111000110
Beta prior: a = 53 b = 33
Likelihood: 0.20659760529408006
Beta posterior: a = 62 b = 39

case 7: 1010010111
Beta prior: a = 62 b = 39
Likelihood: 0.25082265600000003
Beta posterior: a = 68 b = 43

case 8: 11101110110
Beta prior: a = 68 b = 43
Likelihood: 0.2619678932864457
Beta posterior: a = 76 b = 46

case 9: 01000111101
Beta prior: a = 76 b = 46
Likelihood: 0.23609128871506807
Beta posterior: a = 82 b = 51

